In [1]:
import sagemaker

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

bucket = "pipeline-cross-validation"
prefix = "data"
num_class = 8

In [2]:
from sagemaker.workflow.parameters import ParameterString

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.p3.2xlarge"
)
batch_data = ParameterString(
    name="BatchData",
    default_value=f"s3://{bucket}/{prefix}/test/features/",
)

In [3]:
from sagemaker.workflow.retry import (
    StepRetryPolicy, 
    StepExceptionTypeEnum,
    SageMakerJobExceptionTypeEnum,
    SageMakerJobStepRetryPolicy
)

retry_policies=[
    StepRetryPolicy(
        exception_types=[
            StepExceptionTypeEnum.SERVICE_FAULT, 
            StepExceptionTypeEnum.THROTTLING
        ],
        expire_after_mins=5,
        interval_seconds=10,
        backoff_rate=2.0 
    ),
    SageMakerJobStepRetryPolicy(
        exception_types=[SageMakerJobExceptionTypeEnum.RESOURCE_LIMIT],
        expire_after_mins=120,
        interval_seconds=60,
        backoff_rate=2.0
    ),
    SageMakerJobStepRetryPolicy(
        failure_reason_types=[
            SageMakerJobExceptionTypeEnum.INTERNAL_ERROR,
            SageMakerJobExceptionTypeEnum.CAPACITY_ERROR,
        ],
        max_attempts=10,
        interval_seconds=30,
        backoff_rate=2.0
    )
]


In [4]:
hyperparameters = {
    "num_class":num_class,
    "max_depth":"5",
    "eta":"0.2",
    "min_child_weight":"1",
    "objective":"multi:softmax",
    "eval_metric":"mlogloss",
    "num_round":"500",
    "tree_method":"gpu_hist",
    "early_stopping_rounds":"10",
    # "csv_weights": "1"
}

In [5]:
def create_steps(iteration):
    
    # Parameters
    dst_model_path = f"s3://{bucket}/{prefix}/model_train/iter{iteration}"
    train_name = f"train_iter{iteration}"
    model_name = train_name.replace("_", "-")
    dst_test_path = f"s3://{bucket}/{prefix}/output/iter{iteration}"
    transform_name = f"transform_iter{iteration}"
    
    # Training Step
    image_uri = sagemaker.image_uris.retrieve(
        framework="xgboost",
        region=sagemaker_session.boto_region_name,
        version="1.2-2",
        py_version="py3",
        instance_type=training_instance_type,
    )
    
    from sagemaker.estimator import Estimator
    xgb_train = Estimator(
        image_uri=image_uri,
        hyperparameters=hyperparameters,
        instance_type=training_instance_type,
        instance_count=1,
        output_path=dst_model_path,
        role=role,
    )
    
    from sagemaker.inputs import TrainingInput
    from sagemaker.workflow.steps import TrainingStep
    step_train = TrainingStep(
        name=train_name,
        estimator=xgb_train,
        inputs={
            "train": TrainingInput(
                s3_data=f"s3://{bucket}/{prefix}/iter{iteration}/train/",
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=f"s3://{bucket}/{prefix}/iter{iteration}/validation/",
                content_type="text/csv"
            )
        },
        retry_policies=retry_policies,
    )
    
    # Model Step
    from sagemaker.model import Model
    model = Model(
        image_uri=image_uri,
        model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        sagemaker_session=sagemaker_session,
        role=role,
    )
    
    from sagemaker.inputs import CreateModelInput
    from sagemaker.workflow.steps import CreateModelStep
    inputs = CreateModelInput(
        instance_type="ml.m5.large",
    #     accelerator_type="ml.eia1.medium",
    )
    
    step_create_model = CreateModelStep(
        name=model_name,
        model=model,
        inputs=inputs,
    )
    
    # Transform Step
    from sagemaker.transformer import Transformer
    transformer = Transformer(
        model_name=step_create_model.properties.ModelName,
        instance_type="ml.m5.xlarge",
        instance_count=1,
        assemble_with="Line",
        output_path=dst_test_path
    )
    
    from sagemaker.inputs import TransformInput
    from sagemaker.workflow.steps import TransformStep
    step_transform = TransformStep(
        name=transform_name,
        transformer=transformer,
        inputs=TransformInput(
            data=batch_data,
            content_type="csv",
            split_type="Line",
        ),
        retry_policies=retry_policies,
    )
    
    steps = [
        step_train,
        step_create_model,
        step_transform,
    ]
    
    return steps

In [6]:
iterations = 4
whole_steps = []
for iteration in range(iterations):
    steps = create_steps(iteration)
    whole_steps.extend(steps)

In [7]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = f"220709Pipeline2"

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[training_instance_type, batch_data],
    steps=whole_steps,
)


In [8]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [19]:
execution.list_steps()

[{'StepName': 'transform_iter0',
  'StartTime': datetime.datetime(2022, 7, 9, 9, 19, 43, 922000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'AttemptCount': 0,
  'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:us-east-2:173863741403:transform-job/pipelines-rfvats4iv8wu-transform-iter0-gtbzsfdeso'}}},
 {'StepName': 'train-iter0',
  'StartTime': datetime.datetime(2022, 7, 9, 9, 19, 41, 694000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 7, 9, 9, 19, 43, 135000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-2:173863741403:model/pipelines-rfvats4iv8wu-train-iter0-ovzktsordp'}}},
 {'StepName': 'transform_iter2',
  'StartTime': datetime.datetime(2022, 7, 9, 9, 19, 35, 338000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'AttemptCount': 0,
  'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:us-east-2:173863741403:transform-job/pipelines-rfvats4iv8wu-transform-iter2-w3smhrb4x2

In [18]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-2:173863741403:pipeline/220709pipeline2',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-2:173863741403:pipeline/220709pipeline2/execution/rfvats4iv8wu',
 'PipelineExecutionDisplayName': 'execution-1657358129776',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': '220709pipeline2',
  'TrialName': 'rfvats4iv8wu'},
 'CreationTime': datetime.datetime(2022, 7, 9, 9, 15, 29, 692000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 7, 9, 9, 15, 29, 692000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'df595c39-9f4b-4d90-a0d8-631a6ca2cf0f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'df595c39-9f4b-4d90-a0d8-631a6ca2cf0f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '486',
   'date': 'Sat, 09 Jul 2022 09:22:22 GMT'},
  'RetryAttempts': 0}}